# Disponibilização dos arquivos
**Após a criação o índice, ele deve ser gravado em arquivo (inclusive o vocabulário). Favor disponibilizar o link para que eu possa fazer o download do arquivo (via dropbox, por ex). Além de instruções completas para que eu possa abrí­lo. A biblioteca json pode ajudar.**

# Discussão 
A parte mais desafiadora foi entender a estrutura inicial do indexador. Em certas ocasiões tivemos divergências quanto a correta implementação dos métodos e isso levou a muito retrabalho e avanço lento. Por vezes não estava muito claro como proceder com os parâmetros e métodos auxiliares.

# vantagem/desvantagem sobre os modelos comparativos 
Fazer a indexação totalmente pela memória principal poderia facilitar o acesso aos dados, mas pode exigir muito do hardware do computador e fica difícil prever o tamanho do espaço ocupado pois não se sabe quantas palavras em documentos diferentes existem. Já o uso de memória secundária vem como um alívio para memória RAM, porém acessar os dados depois salvos no arquivo binário `.idx` terá um custo associado a lentidão do disco rígido.

# O modelo adotado para a solução
A solução adotada nesse indexador vem buscar um equilíbrio entre uso de memória princípal e secundária. O foco era aproveitar a velocidade da principal e a grande capacidade de armazenamento da secundária, mas sempre intercalando entre elas.

# Possíveis melhorias de eficiência e desempenho
Para deixar o processo mais eficiente seria interessante indexar frases completas ao invés de palavras. Já para deixar diminuir o consumo de memória poderia ser feita uma codificação que abstraísse a informação salva como uma notação em hexadecimal.

# Bibliotecas externas usadas
Claro que houve uso de recurso externo já consolidado do acervo do Python para ajudar nessa tarefa. Ao todo são 5, sendo elas: gc, pickle, nltk, bs4 e string

## gc
Também conhecido como garbage colector, essa biblioteca foi usada para desabilitar o coletor de lixo durante a geração dos arquivos de forma a otimizar o acesso na memória secundária.

## pickle
Usada para escrita nos arquivos `.idx`.

## nltk
Responsável por dividir o texto em tokens tendo `espace` como divisor. Além disso trazer o SnowballStemmer para realizar o stem das palavras filtradas.

## bs4
Responsável por remover marcadores de HTML do meio do texto.

## string
Não menos importante, essa biblioteca tem o papel de apontar todos os símbolos de pontuação através do atributo string.punctuation. A partir dele, foi possível obter todos os sinais que seriam removidos do texto.

# Técnica de streaming adotada

A técnica de streaming se baseia em definir um tamanho máximo para o vetor de ocorrências que funcionará como um Buffer, e caso esse limite seja atingido as ocorrências são salvas em um arquivo.

Na classe `FileIndex` é definido esse limite e no método `add_index_occur` a ocorrência é salva e logo após é realizada a verificação se a lista atingiu seu limite, caso o tenha atingido é chamado o método `save_tmp_occurences` que é resposável por salvar em arquivo o conteúdo da lista.

```python
class FileIndex(Index):
	TMP_OCCURRENCES_LIMIT = 1000000
	# ...
	def add_index_occur(self, entry_dic_index: TermFilePosition, doc_id: int, term_id: int, term_freq: int):
		self.lst_occurrences_tmp.append(TermOccurrence(doc_id, term_id, term_freq))
		if len(self.lst_occurrences_tmp) >= FileIndex.TMP_OCCURRENCES_LIMIT:
			self.save_tmp_occurrences()
```

Quando a lista de ocorrências chega em seu limite o `save_tmp_occurences` é responssável por verificar se já existe outro arquivo salvo, caso exista ele irá ler seu primeiro termo e comparar com o primeiro termo da lista previamente ordenada, a ocorrência que estiver o menor `term_id` será salva no novo arquivo e o processo irá se repetir até que todas as ocorrências da lista e do arquivo estejam salvos, e assim, conseguimos criar um novo arquivo com todas as ocorrências da memória principal e secundário de forma ordenada.

# Estrutura utilizada    
**Qual foi a estrutura do índice adotada?** 

# Resultado obtido    
**Quanto MB de ram cada solução de índice gastou? Em quanto tempo foi realizado a indexação? Qual foi a média por documento?**
